# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [3]:
class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 100)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(100, 10)
    
    def forward(self,x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out
  
model = SimpleNet()

# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)

Next, we can start to train and evaluate!

In [4]:
# train and evaluate
#for epoch in range(NUM_EPOCHS):
    #for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
losses = []
acces = []
test_losses = []
test_acces = []       
for epoch in range(NUM_EPOCHS):
    print('current epoch = %d' % epoch)
    train_loss = 0
    train_acc = 0
    for step, (images, labels) in enumerate(train_loader): 
        
        images = Variable(images.view(-1, 28 * 28))
        labels = Variable(labels)
        
        optimizer.zero_grad()
        
        # forward 
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # backward
        loss.backward()
        optimizer.step()
        
        # record loss
        train_loss += loss.item()
        
        # calculate accuracy
        _, pred = outputs.max(1)
        num_correct = (pred == labels).sum().item()
        acc = num_correct / images.shape[0]
        train_acc += acc
            
    losses.append(train_loss / len(train_loader))
    acces.append(train_acc / len(train_loader))
    
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
    
    # calculate accuracy on test set
    test_loss = 0
    test_acc = 0
    model = model.eval()
    with torch.no_grad():
        for im, label in test_loader:
            if torch.cuda.is_available():
                im = Variable(im.cuda())
                label = Variable(label.cuda())
            else:
                im = Variable(im.view(-1, 28 * 28))
                label = Variable(label)
            output = model(im)
            loss = criterion(output, label)
            
            # record loss
            test_loss += loss.item()
            
            # record accuracy
            _, pred = output.max(1)
            num_correct = (pred == label).sum().item()
            acc = num_correct / im.shape[0]
            test_acc += acc
        
        test_losses.append(test_loss / len(test_loader))
        test_acces.append(test_acc / len(test_loader))
    
    print('epoch: {}, Train Loss: {:.6f}, Train Acc: {:.6f}, Test Loss: {:.6f}, Test Acc: {:.6f}'
          .format(epoch, train_loss / len(train_loader), train_acc / len(train_loader), 
                     test_loss / len(test_loader), test_acc / len(test_loader)))


current epoch = 0
epoch: 0, Train Loss: 0.464605, Train Acc: 0.869775, Test Loss: 0.289795, Test Acc: 0.915164
current epoch = 1
epoch: 1, Train Loss: 0.262911, Train Acc: 0.923544, Test Loss: 0.228454, Test Acc: 0.932492
current epoch = 2
epoch: 2, Train Loss: 0.198483, Train Acc: 0.942675, Test Loss: 0.172545, Test Acc: 0.948017
current epoch = 3
epoch: 3, Train Loss: 0.157924, Train Acc: 0.953659, Test Loss: 0.143415, Test Acc: 0.956530
current epoch = 4
epoch: 4, Train Loss: 0.133839, Train Acc: 0.960253, Test Loss: 0.146784, Test Acc: 0.955729
current epoch = 5
epoch: 5, Train Loss: 0.117692, Train Acc: 0.965178, Test Loss: 0.119403, Test Acc: 0.963341
current epoch = 6
epoch: 6, Train Loss: 0.102711, Train Acc: 0.968767, Test Loss: 0.114211, Test Acc: 0.964143
current epoch = 7
epoch: 7, Train Loss: 0.093803, Train Acc: 0.971989, Test Loss: 0.123596, Test Acc: 0.962640
current epoch = 8
epoch: 8, Train Loss: 0.086156, Train Acc: 0.973658, Test Loss: 0.109386, Test Acc: 0.964944
c

#### Q5:
Please print the training and testing accuracy.

In [5]:
for epoch in range(10):
    print('epoch: {}, Train Acc: {:.6f}, Test Acc: {:.6f}'.format(epoch, acces[epoch], test_acces[epoch]))

epoch: 0, Train Acc: 0.869775, Test Acc: 0.915164
epoch: 1, Train Acc: 0.923544, Test Acc: 0.932492
epoch: 2, Train Acc: 0.942675, Test Acc: 0.948017
epoch: 3, Train Acc: 0.953659, Test Acc: 0.956530
epoch: 4, Train Acc: 0.960253, Test Acc: 0.955729
epoch: 5, Train Acc: 0.965178, Test Acc: 0.963341
epoch: 6, Train Acc: 0.968767, Test Acc: 0.964143
epoch: 7, Train Acc: 0.971989, Test Acc: 0.962640
epoch: 8, Train Acc: 0.973658, Test Acc: 0.964944
epoch: 9, Train Acc: 0.975394, Test Acc: 0.967047
